In [ ]:
import json
import pandas as pd

from datetime import datetime

In [ ]:
# dataset home page: https://brasil.io/dataset/covid19/caso
# csv listed at: https://data.brasil.io/dataset/covid19/_meta/list.html
data = pd.read_csv("https://data.brasil.io/dataset/covid19/caso_full.csv.gz")

# 1. Inspect the data

In [ ]:
data.info()

In [ ]:
data['new_confirmed'].sum(), data['new_deaths'].sum()

In [ ]:
data.groupby(['place_type']).size()

In [ ]:
data.groupby(['is_repeated']).size()

In [ ]:
data.loc[0:10, ["date", "city", "state", "place_type", "is_repeated", "last_available_date", 
               "last_available_confirmed", "last_available_deaths", "new_confirmed", "new_deaths"]]

In [ ]:
data[data["state"] == "SP"].loc[0:, ["date", "city", "state", "last_available_confirmed"]]

In [ ]:
data[data["place_type"] == "state"].groupby(["date"])["new_confirmed", "new_deaths"].sum()

In [ ]:
data[(data["state"] == "CE") & (data["place_type"] == "state")].groupby(["date"])["new_deaths", "last_available_deaths", "new_confirmed", 
                                              "last_available_confirmed"].sum()

In [ ]:
data.loc[(data["new_deaths"] < 0) | (data["new_confirmed"] < 0), 
         ["date", "city", "last_available_date", "last_available_confirmed", "last_available_deaths",  "new_confirmed", "new_deaths"]]

# 3. Create `.json` with summarized data

In [ ]:
data.rename(columns={"last_available_deaths": "deaths", "last_available_confirmed": "confirmed"}, inplace=True)

In [ ]:
data_summary = data[data["place_type"] == "state"].groupby(['state', 'date'])['deaths', 'confirmed'].sum().reset_index()

data_summary["deaths"] = data_summary["deaths"].astype(int)
data_summary["confirmed"] = data_summary["confirmed"].astype(int)

In [ ]:
data_summary.set_index("date", inplace=True)

In [ ]:
data_summary.head(3)

In [ ]:
data_summary.tail(10)

In [ ]:
data_summary.loc[data_summary["state"] == "SP", "confirmed"].values

In [ ]:
data_summary.loc[data_summary["state"] == "SP", "deaths"].values

In [ ]:
output = [{"state_name": "Acre", "state_code": "AC", "region": "N", "confirmed":[], "deaths":[]},
          {"state_name": "Alagoas", "state_code": "AL", "region": "NE", "confirmed":[], "deaths":[]},
          {"state_name": "Amazonas", "state_code": "AM", "region": "N", "confirmed":[], "deaths":[]},
          {"state_name": "Amapá", "state_code": "AP", "region": "N", "confirmed":[], "deaths":[]},
          {"state_name": "Bahia", "state_code": "BA", "region": "NE", "confirmed":[], "deaths":[]},
          {"state_name": "Ceará", "state_code": "CE", "region": "NE", "confirmed":[], "deaths":[]},
          {"state_name": "Distrito Federal", "state_code": "DF", "region": "CO", "confirmed":[], "deaths":[]},
          {"state_name": "Espírito Santo", "state_code": "ES", "region": "SE", "confirmed":[], "deaths":[]},
          {"state_name": "Goiás", "state_code": "GO", "region": "CO", "confirmed":[], "deaths":[]},
          {"state_name": "Maranhão", "state_code": "MA", "region": "NE", "confirmed":[], "deaths":[]},
          {"state_name": "Minas Gerais", "state_code": "MG", "region": "SE", "confirmed":[], "deaths":[]},
          {"state_name": "Mato Grosso do Sul", "state_code": "MS", "region": "CO", "confirmed":[], "deaths":[]},
          {"state_name": "Mato Grosso", "state_code": "MT", "region": "CO", "confirmed":[], "deaths":[]},
          {"state_name": "Pará", "state_code": "PA", "region": "N", "confirmed":[], "deaths":[]},
          {"state_name": "Paraíba", "state_code": "PB", "region": "NE", "confirmed":[], "deaths":[]},
          {"state_name": "Pernambuco", "state_code": "PE", "region": "NE", "confirmed":[], "deaths":[]},
          {"state_name": "Piauí", "state_code": "PI", "region": "NE", "confirmed":[], "deaths":[]},
          {"state_name": "Paraná", "state_code": "PR", "region": "S", "confirmed":[], "deaths":[]},
          {"state_name": "Rio de Janeiro", "state_code": "RJ", "region": "SE", "confirmed":[], "deaths":[]},
          {"state_name": "Rio Grande do Norte", "state_code": "RN", "region": "NE", "confirmed":[], "deaths":[]},
          {"state_name": "Rondônia", "state_code": "RO", "region": "N", "confirmed":[], "deaths":[]},
          {"state_name": "Roraima", "state_code": "RR", "region": "N", "confirmed":[], "deaths":[]},
          {"state_name": "Rio Grande do Sul", "state_code": "RS", "region": "S", "confirmed":[], "deaths":[]},
          {"state_name": "Santa Catarina", "state_code": "SC", "region": "S", "confirmed":[], "deaths":[]},          
          {"state_name": "Sergipe", "state_code": "SE", "region": "NE", "confirmed":[], "deaths":[]},          
          {"state_name": "São Paulo", "state_code": "SP", "region": "SE", "confirmed":[], "deaths":[]},          
          {"state_name": "Tocantins", "state_code": "TO", "region": "N", "confirmed":[], "deaths":[]},                    
         ]

In [ ]:
for item in output:
    code = item["state_code"]
    
    dates_conf = data_summary.loc[data_summary["state"] == code, "confirmed"].index
    confirmed = data_summary.loc[data_summary["state"] == code, "confirmed"].values
    dates_deaths = data_summary.loc[data_summary["state"] == code, "deaths"].index
    deaths = data_summary.loc[data_summary["state"] == code, "deaths"].values
        
    item["confirmed"] = [[date, int(value)] for date, value in zip(dates_conf, confirmed)]
    item["deaths"] = [[date, int(value)] for date, value in zip(dates_deaths, deaths)]

In [ ]:
print(output)

In [ ]:
with open('state_summary.json', 'w') as f:
    json.dump(output, f, indent=2)

# 4. Update `index.html`

In [ ]:
update_timestamp = datetime.strftime(datetime.today(), "%Y-%m-%d %H:%M")

In [ ]:
html_str = f'''
<!DOCTYPE html>
<html lang="en-US">
  <head>
    <meta charset="UTF-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />

    <!-- Begin Jekyll SEO tag v2.6.1 -->
    <title>COVID-19 no Brasil</title>
    <meta name="generator" content="Jekyll v3.8.5" />
    <meta property="og:title" content="covid-19-brasil" />
    <meta property="og:locale" content="en_US" />
    <link rel="canonical" href="https://lucasrla.github.io/covid-19-brasil/" />
    <meta
      property="og:url"
      content="https://lucasrla.github.io/covid-19-brasil/"
    />
    <meta property="og:site_name" content="covid-19-brasil" />
    <script type="application/ld+json">
      {{
        "headline": "Evolução do COVID-19 no Brasil",
        "@type": "WebSite",
        "url": "https://lucasrla.github.io/covid-19-brasil/",
        "name": "covid-19-brasil",
        "author": "Lucas Amaro",
        "@context": "https://schema.org"
      }}
    </script>
    <!-- End Jekyll SEO tag -->

    <link
      rel="stylesheet"
      href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css"
      integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh"
      crossorigin="anonymous"
    />
  </head>
  <body>
    <div class="container-xl">
      <div class="mt-5 mb-5">
        <h1>COVID-19 Casos e Mortes no Brasil</h1>
        <h2>Evolução por Estado</h2>
        <h5>Última atualização: {update_timestamp}</h5>
      </div>

      <div id="observablehq-content" class="mt-5">
        <div id="y_type" class="mb-4"></div>
        <div id="pivot_n" class="mb-4"></div>
        <div id="num_days" class="mb-4"></div>
        <div id="doubling_periods" class="mb-4"></div>
        <div id="selected_regions" class="mb-4"></div>
        <div id="chart" width="100%"></div>
        <div id="swatch" class="mb-5"></div>
      </div>

      <p>
        Fonte:
        <a href="https://brasil.io/dataset/covid19/caso" target="_blank"
          >Secretarias de Saúde das Unidades Federativas, dados tratados por
          Álvaro Justen / Brasil.IO</a
        >
      </p>

      <div>
        Avisos:
        <ul>
          <li>
            A cada dia, casos e mortes reportadas pelas Secretarias de Saúde
            (que utilizamos acima) podem estar em desincronia com os dados
            informados pelo Ministério da Saúde em suas coletivas de imprensa.
            Esses eventuais descasamentos entre as fontes são inevitáveis,
            porém, felizmente, são temporários. É esperado que todas elas
            convirjam.
          </li>
          <li>
            Cada cidade tem sua própria metodologia e desafios (que variam no
            tempo) para testar casos e para informar mortes suspeitas. Por isso,
            tanto casos quanto mortes estão certamente sendo subestimados acima.
          </li>
          <li>
            Caso tenha interesse nos dados de óbitos suspeitos e confirmados,
            visite este
            <a
              href="https://transparencia.registrocivil.org.br/especial-covid"
              target="_blank"
              >site do Portal da Transparência</a
            >. Lá estão os dados diários de todos os Cartórios de Registro Civil
            do Brasil.
          </li>
        </ul>
      </div>

      <hr />

      <p>
        Código-fonte desta página está disponível em
        <a href="https://github.com/lucasrla/covid-19-brasil" target="_blank"
          >https://github.com/lucasrla/covid-19-brasil</a
        >
      </p>
    </div>

    <script type="module">
      import notebook from "https://api.observablehq.com/@lucasrla/covid-19-casos-e-mortes-no-brasil.js?v=3";

      const target = document.querySelector("#observablehq-content");

      const renders = {{
        "viewof y_type": "#y_type",
        "viewof pivot_n": "#pivot_n",
        "viewof num_days": "#num_days",
        "viewof doubling_periods": "#doubling_periods",
        "viewof selected_regions": "#selected_regions",
        swatch: "#swatch",
        chart: "#chart",
      }};

      import {{
        Inspector,
        Runtime,
      }} from "https://cdn.jsdelivr.net/npm/@observablehq/runtime@4/dist/runtime.js";

      console.log(renders);

      //ref1: https://visionscarto.net/observable-jekyll/
      //ref2: https://talk.observablehq.com/t/embedding-with-runtime-load-seems-to-no-longer-load-reference-dependencies/1617/4
      const runtime = new Runtime();
      const main = runtime.module(notebook, (name) => {{
        const elmSelector = renders[name];
        if (elmSelector) {{
          return new Inspector(document.querySelector(elmSelector));
        }}
      }});
    </script>
  </body>
</html>
'''

In [ ]:
with open("index.html", "w") as f:
    f.write(html_str)